# Descomposición CUR

### Carga de Librerías (Python 3.7)

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
from numpy.linalg import svd
from numpy.linalg import linalg
from time import time
import random

### Importar archivo con calificaciones (ratings)

In [6]:
# Conjunto de datos con un millón de calificaciones (cifra redondeada)
ratings = pd.read_csv(r'ratings_1m.csv')

# Separar conjunto 75% entrenamiento y 25% validación, se usa train_test_split de sklearn
train, test = train_test_split(ratings,test_size=0.25)

### Mapeo de Usuarios y de Películas

In [7]:
# El mapeo lo que hace es tener un id consecutivo y único de usuarios y de películas
# La tabla original de películas no tiene id consecutivos

lista_pelic = ratings['movieId'].unique()
num_pelic = len(lista_pelic)
mapeo_pelic = {}
for n, m in enumerate(lista_pelic):
    mapeo_pelic[m] = n
lista_usuarios = ratings['userId'].unique()
num_usuarios = len(lista_usuarios)
mapeo_usuarios = {}
for n, m in enumerate(lista_usuarios):
    mapeo_usuarios[m] = n

### Creación de Matriz de usuarios y películas

In [8]:
matriz_uspl = np.zeros([num_usuarios, num_pelic])
for index, row in ratings.iterrows():
    matriz_uspl[mapeo_usuarios[row['userId']], mapeo_pelic[row['movieId']]] = int(row['rating'])

# Se guarda una copia como Data Frame
matriz_df = pd.DataFrame(matriz_uspl)    

### Descomposición CUR

In [9]:
# Se contabiliza el tiempo de procesamiento para la Descomposición CUR
tiempo_inicio = time()

# Cálculo de los promedios de las calificaciones de cada usuario
prom_usr = matriz_uspl.sum(axis=1)
aux = Counter(matriz_uspl.nonzero()[0])
for i in range(num_usuarios):
    if i in aux.keys():
        prom_usr[i] = prom_usr[i]/aux[i]
    else:
        prom_usr[i] = 0

# Cálculo de los promedios de las calificaciones por película
prom_pelic = matriz_uspl.T.sum(axis=1)
aux = Counter(matriz_uspl.T.nonzero()[0])
for i in range(num_pelic):
    if i in aux.keys():
        prom_pelic[i] = prom_pelic[i]/aux[i]
    else:
        prom_pelic[i] = 0

# Se calculan las probabilidades de seleccionar columnas y renglones
total_norm =  np.linalg.norm(matriz_uspl)  # Calcula la norma de la matriz de usuarios y películas
col_norm =  np.linalg.norm(matriz_uspl,axis = 0) #calcula la norma del vector columna
row_norm =  np.linalg.norm(matriz_uspl,axis = 1) #calcula la norma del vector renglón

for i in range(num_pelic):
    col_norm[i] = (col_norm[i]/total_norm)**2
    
for i in range(num_usuarios):
    row_norm[i] = (row_norm[i]/total_norm)**2

# Con las probabilidades calculadas se seleccionan las columnas aleatoriamente y sin duplicar

# Para el conjunto de datos de 100,000 calificaciones se reducirá a 90 columnas
# Se hicieron pruebas con números más grandes, la SVD no convergía
# Para el conjunto de datos de 1,000,000 calificaciones se reducirá a 800 columnas

c=800
col_selec = []
C = np.zeros([num_usuarios,c])
col_selec = random.sample(range(num_pelic-1),800)
    
for i in range(800):
    x = col_selec[i]
    p = col_norm[x]
    d = np.sqrt(c*p)
    C[:,i] = matriz_df.loc[:,x]/d
    i = i + 1

# Con las probabilidades calculadas se seleccionan los renglones aleatoriamente y sin duplicar
# Para el conjunto de datos de 100,000 calificaciones se reducirá a 90 renglones
    
r=800
ren_selec = []
R = np.zeros([num_pelic,r])
ren_selec = random.sample(range(num_usuarios-1),800)

for i in range(800):
    x = ren_selec[i]
    p = row_norm[x]
    d = np.sqrt(r*p)
    R[:,i] = matriz_df.T.loc[:,x]/d
    i = i + 1
    
# La matriz U es construida de W por la pseudoinversa de Moore-Penrose

W = C[ren_selec,:]
W1, W_cur, W2 = svd(W)
W_cur = np.diag(W_cur)
for i in range(W_cur.shape[0]):
    W_cur[i][i] = 1/W_cur[i][i]
U = np.dot(np.dot(W2.T, W_cur**2), W1.T)
CUR = np.dot(np.dot(C, U), R.T)

CUR_df = pd.DataFrame
# El rango de calificaciones va de 0 a 5, sin embargo, hay casos menores al primero
# y mayores al segundo. Los menores a 0 se cambian por 0 y los mayores a 5 por 5

for i in range(CUR.shape[0]):
    for j in range(CUR.shape[1]):
        if CUR[i,j] > 5:
            CUR[i,j] = 5
        elif CUR[i,j] < 0:
            CUR[i,j] = 0
tiempo_fin = time()

#### Función para cálculo de Error Cuadrático Medio (RMSE)

In [10]:
# Error Cuadrático Medio (RMSE)
def ECM(pred,original):
    N = pred.shape[0] # filas
    M = pred.shape[1] # columnas
    cur_sum = np.sum(np.square(pred-original))
    return np.sqrt(cur_sum/(N*M))

#### Función para cálculo de Coeficiente de Correlación de Spearman (rho)

En estadística, el coeficiente de correlación de Spearman, $\rho$  es una medida de correlación
El estadístico $\rho$ viene dado por la expresión:
$$ \rho = 1 - \frac{6 \Sigma D^2}{N(N^2-1)} $$
donde D es la diferencia entre los correspondientes estadísticos de orden x-y. N es el número de parejas de datos

La correlación de Spearman evalúa relaciones monótonas (sean lineales o no). 

En matemáticas una función entre conjuntos ordenados se dice monótona si conserva el orden dado

Intiuitivamente, la correlación de Spearman entre dos observaciones será alta (correlación cercana a 1) cuando las observaciones tengan un rango similar

In [11]:
# Coeficiente de correlación de Spearman (rho)
def rho(pred, original):
    num = np.sum(np.square(pred - original))
    n = pred.shape[0]*pred.shape[1]
    den = n*(n*n-1)
    return 1 - (6*num)/den

### Resultados Descomposición CUR

In [12]:
print("Error Cuadrático Medio", ECM(CUR, matriz_uspl))
print("Coeficiente de correlación de Spearman", rho(CUR, matriz_uspl))
print("Tiempo de procesamiento en segundos", tiempo_fin - tiempo_inicio)

Error Cuadrático Medio 3.199002524365957
Coeficiente de correlación de Spearman 0.9999999999998774
Tiempo de procesamiento en segundos 28.94983696937561


### Descomposición en Valores Singulares (SVD)

#### Cálculo de Eigen Valores y Eigen Vectores

In [13]:
tiempo_inicial_SVD = time()
m1 = np.dot(matriz_uspl, matriz_uspl.T)
eigenValues, eigenVectors = linalg.eig(m1)
eigenValues = eigenValues.real
eigenVectors = eigenVectors.real
eigen_map = dict()
for i in range(len(eigenValues)):
	eigenValues[i] = round(eigenValues[i], 4)
for i in range(len(eigenValues)):
    if eigenValues[i] != 0:
        eigen_map[eigenValues[i]] = eigenVectors[:, i]

eigenValues = sorted(eigen_map.keys(), reverse=True)
U = np.zeros_like(eigenVectors)
for i in range(len(eigenValues)):
    U[:,i] = eigen_map[eigenValues[i]]
U = U[:,:len(eigenValues)]

#### Cálculo de Sigma

In [14]:
Sigma = np.diag([i**0.5 for i in eigenValues])

#### Cálculo de V

In [15]:
m2 = np.dot(matriz_uspl.T, matriz_uspl)
eigenValues, eigenVectors = linalg.eig(m2)
eigenValues = eigenValues.real
eigenVectors = eigenVectors.real
for i in range(len(eigenValues)):
	eigenValues[i] = round(eigenValues[i], 4)
eigen_map = dict()
for i in range(len(eigenValues)):
    if eigenValues[i] != 0:
        eigen_map[eigenValues[i]] = eigenVectors[:, i]
eigenValues = sorted(eigen_map.keys(), reverse=True)
V = np.zeros_like(eigenVectors)
for i in range(len(eigenValues)):
    V[:,i] = eigen_map[eigenValues[i]]
V = V[:,:len(eigenValues)]
V = V.T

t_aux_1 = time()

#### Cálculo SVD con 100% de energía

In [16]:
shape = Sigma.shape
SVD100 = np.dot(np.dot(U,Sigma),V)
t_final_SVD100 = time()

### Resultados SVD 100% de energía

In [17]:
print("Error Cuadrático Medio SVD 100%", ECM(SVD100, matriz_uspl))
print("Coeficiente de correlación de Spearman", rho(SVD100, matriz_uspl))
print("Tiempo de procesamiento en segundos SVD 100%", t_final_SVD100 - tiempo_inicial_SVD)

Error Cuadrático Medio SVD 100% 0.9931917152158468
Coeficiente de correlación de Spearman 0.9999999999999882
Tiempo de procesamiento en segundos SVD 100% 270.87736439704895


#### Cálculo SVD con retención del 90% de la energía

In [18]:
# Una regla de dedo es retener suficientes valores singulares para representar el 90% de energía

suma_total = 0
dimensions = Sigma.shape[0]

# Calcula el cuadrado de la suma de todas las diagonales
for i in range(dimensions):
    suma_total = suma_total + np.square(Sigma[i,i])	
retencion = suma_total
while dimensions > 0:
	retencion = retencion - np.square(Sigma[dimensions-1,dimensions-1])
	if retencion/suma_total < 0.9:	# Retención del 90% de energía
		break
	else:
		U = U[:,:-1:]
		V = V[:-1,:]
		Sigma = Sigma[:,:-1]
		Sigma = Sigma[:-1,:]
		dimensions = dimensions - 1	# reducción de dimensionalidad
t_aux_2 = time()

SVD90 = np.dot(np.dot(U,Sigma),V)
t_final_SVD90 = time()

### Resultados SVD con retención del 90% de la energía

In [19]:
print("Error Cuadrático Medio SVD 90%", ECM(SVD90, matriz_uspl))
print("Coeficiente de correlación de Spearman SVD 90%", rho(SVD90, matriz_uspl))
print("Tiempo de procesamiento en segundos SVD 90%",t_final_SVD90 - t_aux_2 + t_aux_1 - tiempo_inicial_SVD)

Error Cuadrático Medio SVD 90% 0.9613340042443517
Coeficiente de correlación de Spearman SVD 90% 0.9999999999999889
Tiempo de procesamiento en segundos SVD 90% 266.4422335624695


### Pruebas con distintos tamaños de matrices para Descomposición CUR

In [20]:
# Pruebas con matrices de distintos tamaños (3,500 columnas y renglones)

c=3500
col_selec = []
C = np.zeros([num_usuarios,c])
col_selec = random.sample(range(num_pelic-1),3500)
    
for i in range(3500):
    x = col_selec[i]
    p = col_norm[x]
    d = np.sqrt(c*p)
    C[:,i] = matriz_df.loc[:,x]/d
    i = i + 1
   
r=3500
ren_selec = []
R = np.zeros([num_pelic,r])
ren_selec = random.sample(range(num_usuarios-1),3500)

for i in range(3500):
    x = ren_selec[i]
    p = row_norm[x]
    d = np.sqrt(r*p)
    R[:,i] = matriz_df.T.loc[:,x]/d
    i = i + 1
    
# La matriz U es construida de W por la pseudoinversa de Moore-Penrose

W = C[ren_selec,:]
W1, W_cur, W2 = svd(W)
W_cur = np.diag(W_cur)
for i in range(W_cur.shape[0]):
    W_cur[i][i] = 1/W_cur[i][i]
U = np.dot(np.dot(W2.T, W_cur**2), W1.T)
CUR_3500 = np.dot(np.dot(C, U), R.T)

# El rango de calificaciones va de 0 a 5, sin embargo, hay casos menores al primero
# y mayores al segundo. Los menores a 0 se cambian por 0 y los mayores a 5 por 5

for i in range(CUR_3500.shape[0]):
    for j in range(CUR_3500.shape[1]):
        if CUR_3500[i,j] > 5:
            CUR_3500[i,j] = 5
        elif CUR_3500[i,j] < 0:
            CUR_3500[i,j] = 0
            
# Pruebas con matrices de distintos tamaños (3,000 columnas y renglones)

c=3000
col_selec = []
C = np.zeros([num_usuarios,c])
col_selec = random.sample(range(num_pelic-1),3000)
    
for i in range(3000):
    x = col_selec[i]
    p = col_norm[x]
    d = np.sqrt(c*p)
    C[:,i] = matriz_df.loc[:,x]/d
    i = i + 1
   
r=3000
ren_selec = []
R = np.zeros([num_pelic,r])
ren_selec = random.sample(range(num_usuarios-1),3000)

for i in range(3000):
    x = ren_selec[i]
    p = row_norm[x]
    d = np.sqrt(r*p)
    R[:,i] = matriz_df.T.loc[:,x]/d
    i = i + 1
    
# La matriz U es construida de W por la pseudoinversa de Moore-Penrose

W = C[ren_selec,:]
W1, W_cur, W2 = svd(W)
W_cur = np.diag(W_cur)
for i in range(W_cur.shape[0]):
    W_cur[i][i] = 1/W_cur[i][i]
U = np.dot(np.dot(W2.T, W_cur**2), W1.T)
CUR_3000 = np.dot(np.dot(C, U), R.T)

# El rango de calificaciones va de 0 a 5, sin embargo, hay casos menores al primero
# y mayores al segundo. Los menores a 0 se cambian por 0 y los mayores a 5 por 5

for i in range(CUR_3000.shape[0]):
    for j in range(CUR_3000.shape[1]):
        if CUR_3000[i,j] > 5:
            CUR_3000[i,j] = 5
        elif CUR_3000[i,j] < 0:
            CUR_3000[i,j] = 0
            
#print("\nDescomposición CUR matriz 3000 columnas y renglones")
#print("Error Cuadrático Medio", ECM(CUR_3000, matriz_uspl))
#print("Coeficiente de correlación de Spearman", rho(CUR_3000, matriz_uspl))

# Pruebas con matrices de distintos tamaños (1,200 columnas y renglones)

c=1200
col_selec = []
C = np.zeros([num_usuarios,c])
col_selec = random.sample(range(num_pelic-1),1200)
    
for i in range(1200):
    x = col_selec[i]
    p = col_norm[x]
    d = np.sqrt(c*p)
    C[:,i] = matriz_df.loc[:,x]/d
    i = i + 1
   
r=1200
ren_selec = []
R = np.zeros([num_pelic,r])
ren_selec = random.sample(range(num_usuarios-1),1200)

for i in range(1200):
    x = ren_selec[i]
    p = row_norm[x]
    d = np.sqrt(r*p)
    R[:,i] = matriz_df.T.loc[:,x]/d
    i = i + 1
    
# La matriz U es construida de W por la pseudoinversa de Moore-Penrose

W = C[ren_selec,:]
W1, W_cur, W2 = svd(W)
W_cur = np.diag(W_cur)
for i in range(W_cur.shape[0]):
    W_cur[i][i] = 1/W_cur[i][i]
U = np.dot(np.dot(W2.T, W_cur**2), W1.T)
CUR_1200 = np.dot(np.dot(C, U), R.T)

# El rango de calificaciones va de 0 a 5, sin embargo, hay casos menores al primero
# y mayores al segundo. Los menores a 0 se cambian por 0 y los mayores a 5 por 5

for i in range(CUR_1200.shape[0]):
    for j in range(CUR_1200.shape[1]):
        if CUR_1200[i,j] > 5:
            CUR_1200[i,j] = 5
        elif CUR_1200[i,j] < 0:
            CUR_1200[i,j] = 0
            
#print("\nDescomposición CUR matriz 1200 columnas y renglones")
#print("Error Cuadrático Medio", ECM(CUR_1200, matriz_uspl))
#print("Coeficiente de correlación de Spearman", rho(CUR_1200, matriz_uspl))


# Pruebas con matrices de distintos tamaños (400 columnas y renglones)

c=400
col_selec = []
C = np.zeros([num_usuarios,c])
col_selec = random.sample(range(num_pelic-1),400)
    
for i in range(400):
    x = col_selec[i]
    p = col_norm[x]
    d = np.sqrt(c*p)
    C[:,i] = matriz_df.loc[:,x]/d
    i = i + 1
   
r=400
ren_selec = []
R = np.zeros([num_pelic,r])
ren_selec = random.sample(range(num_usuarios-1),400)

for i in range(400):
    x = ren_selec[i]
    p = row_norm[x]
    d = np.sqrt(r*p)
    R[:,i] = matriz_df.T.loc[:,x]/d
    i = i + 1
    
# La matriz U es construida de W por la pseudoinversa de Moore-Penrose

W = C[ren_selec,:]
W1, W_cur, W2 = svd(W)
W_cur = np.diag(W_cur)
for i in range(W_cur.shape[0]):
    W_cur[i][i] = 1/W_cur[i][i]
U = np.dot(np.dot(W2.T, W_cur**2), W1.T)
CUR_2 = np.dot(np.dot(C, U), R.T)

# El rango de calificaciones va de 0 a 5, sin embargo, hay casos menores al primero
# y mayores al segundo. Los menores a 0 se cambian por 0 y los mayores a 5 por 5

for i in range(CUR_2.shape[0]):
    for j in range(CUR_2.shape[1]):
        if CUR_2[i,j] > 5:
            CUR_2[i,j] = 5
        elif CUR_2[i,j] < 0:
            CUR_2[i,j] = 0
            
#print("\nDescomposición CUR matriz 400 columnas y renglones")
#print("Error Cuadrático Medio", ECM(CUR_2, matriz_uspl))
#print("Coeficiente de correlación de Spearman", rho(CUR_2, matriz_uspl))

# Pruebas con matrices de distintos tamaños (50 columnas y renglones)

c=50
col_selec = []
C = np.zeros([num_usuarios,c])
col_selec = random.sample(range(num_pelic-1),50)
    
for i in range(50):
    x = col_selec[i]
    p = col_norm[x]
    d = np.sqrt(c*p)
    C[:,i] = matriz_df.loc[:,x]/d
    i = i + 1
   
r=50
ren_selec = []
R = np.zeros([num_pelic,r])
ren_selec = random.sample(range(num_usuarios-1),50)

for i in range(50):
    x = ren_selec[i]
    p = row_norm[x]
    d = np.sqrt(r*p)
    R[:,i] = matriz_df.T.loc[:,x]/d
    i = i + 1
    
# La matriz U es construida de W por la pseudoinversa de Moore-Penrose

W = C[ren_selec,:]
W1, W_cur, W2 = svd(W)
W_cur = np.diag(W_cur)
for i in range(W_cur.shape[0]):
    W_cur[i][i] = 1/W_cur[i][i]
U = np.dot(np.dot(W2.T, W_cur**2), W1.T)
CUR_3 = np.dot(np.dot(C, U), R.T)

# El rango de calificaciones va de 0 a 5, sin embargo, hay casos menores al primero
# y mayores al segundo. Los menores a 0 se cambian por 0 y los mayores a 5 por 5

for i in range(CUR_3.shape[0]):
    for j in range(CUR_3.shape[1]):
        if CUR_3[i,j] > 5:
            CUR_3[i,j] = 5
        elif CUR_3[i,j] < 0:
            CUR_3[i,j] = 0
            
#print("\nDescomposición CUR matriz 50 columnas y renglones")
#print("Error Cuadrático Medio", ECM(CUR_3, matriz_uspl))
#print("Coeficiente de correlación de Spearman", rho(CUR_3, matriz_uspl))

# Pruebas con matrices de distintos tamaños (10 columnas y renglones)

c=10
col_selec = []
C = np.zeros([num_usuarios,c])
col_selec = random.sample(range(num_pelic-1),10)
    
for i in range(10):
    x = col_selec[i]
    p = col_norm[x]
    d = np.sqrt(c*p)
    C[:,i] = matriz_df.loc[:,x]/d
    i = i + 1
   
r=10
ren_selec = []
R = np.zeros([num_pelic,r])
ren_selec = random.sample(range(num_usuarios-1),10)

for i in range(10):
    x = ren_selec[i]
    p = row_norm[x]
    d = np.sqrt(r*p)
    R[:,i] = matriz_df.T.loc[:,x]/d
    i = i + 1
    
# La matriz U es construida de W por la pseudoinversa de Moore-Penrose

W = C[ren_selec,:]
W1, W_cur, W2 = svd(W)
W_cur = np.diag(W_cur)
for i in range(W_cur.shape[0]):
    W_cur[i][i] = 1/W_cur[i][i]
U = np.dot(np.dot(W2.T, W_cur**2), W1.T)
CUR_4 = np.dot(np.dot(C, U), R.T)

# El rango de calificaciones va de 0 a 5, sin embargo, hay casos menores al primero
# y mayores al segundo. Los menores a 0 se cambian por 0 y los mayores a 5 por 5

for i in range(CUR_4.shape[0]):
    for j in range(CUR_4.shape[1]):
        if CUR_4[i,j] > 5:
            CUR_4[i,j] = 5
        elif CUR_4[i,j] < 0:
            CUR_4[i,j] = 0

C:\Users\IkerOlivo\Anaconda3\lib\site-packages\ipykernel_launcher.py:280: RuntimeWarning: divide by zero encountered in double_scalars


### Comparación Error Cuadrático Medio | Descomposición CUR

In [22]:
print("3500 Columnas y Renglones", ECM(CUR_3500, matriz_uspl))
print("3000 Columnas y Renglones", ECM(CUR_3000, matriz_uspl))
print("1200 Columnas y Renglones", ECM(CUR_1200, matriz_uspl))
print("800 Columnas y Renglones", ECM(CUR, matriz_uspl))
print("400 Columnas y Renglones", ECM(CUR_2, matriz_uspl))
print("50 Columnas y Renglones", ECM(CUR_3, matriz_uspl))
print("10 Columnas y Renglones", ECM(CUR_4, matriz_uspl))

3500 Columnas y Renglones 3.369111258521977
3000 Columnas y Renglones 3.369603640481926
1200 Columnas y Renglones 3.284544624111253
800 Columnas y Renglones 3.199002524365957
400 Columnas y Renglones 3.0918480877407024
50 Columnas y Renglones 1.851972265433165
10 Columnas y Renglones nan


### Comparación Coeficiente de Correlación de Spearman $\rho$ | Descomposición CUR

In [23]:
print("\n Comparación Coeficiente de correlación de Spearman")
print("3500 Columnas y Renglones", rho(CUR_3500, matriz_uspl))
print("3000 Columnas y Renglones", rho(CUR_3000, matriz_uspl))
print("1200 Columnas y Renglones", rho(CUR_1200, matriz_uspl))
print("800 Columnas y Renglones", rho(CUR, matriz_uspl))
print("400 Columnas y Renglones", rho(CUR_2, matriz_uspl))
print("50 Columnas y Renglones", rho(CUR_3, matriz_uspl))
print("10 Columnas y Renglones", rho(CUR_4, matriz_uspl))


 Comparación Coeficiente de correlación de Spearman
3500 Columnas y Renglones 0.9999999999998641
3000 Columnas y Renglones 0.999999999999864
1200 Columnas y Renglones 0.9999999999998708
800 Columnas y Renglones 0.9999999999998774
400 Columnas y Renglones 0.9999999999998855
50 Columnas y Renglones 0.9999999999999589
10 Columnas y Renglones nan
